In [1]:
import calendar
import locale
from datetime import datetime

import pandas as pd
import requests

# Configurar localidade para português do Brasil
locale.setlocale(locale.LC_TIME, "Portuguese_Brazil.1252")


def obter_feriados_brasil(ano=2024):
    """
    Retorna um DataFrame com feriados nacionais do Brasil para o ano especificado.
    Colunas: ['date', 'localName', 'weekday', 'month_name']
    """
    url = f"https://date.nager.at/api/v3/publicholidays/{ano}/BR"
    response = requests.get(url)
    feriados_json = response.json()

    df = pd.DataFrame(feriados_json)
    df["date"] = pd.to_datetime(df["date"])
    df["weekday"] = df["date"].dt.weekday  # Segunda=0, Domingo=6
    df["month_name"] = df["date"].dt.strftime("%B").str.capitalize()
    return df[["date", "localName", "weekday", "month_name"]]


def obter_clima_historico_rio(inicio="2024-01-01", fim="2024-07-31"):
    """
    Retorna um DataFrame com dados históricos do clima para o Rio de Janeiro.
    Colunas: ['date', 'weathercode', 'temp_mean', 'month', 'month_name']
    """
    # Coordenadas do Rio de Janeiro
    lat, lon = -22.9068, -43.1729
    base_url = "https://archive-api.open-meteo.com/v1/era5"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": inicio,
        "end_date": fim,
        "daily": "weathercode,temperature_2m_mean",
        "timezone": "America/Sao_Paulo",
    }
    response = requests.get(base_url, params=params)
    data_json = response.json()

    df = pd.DataFrame({
        "date": pd.to_datetime(data_json["daily"]["time"]),
        "weathercode": data_json["daily"]["weathercode"],
        "temp_mean": data_json["daily"]["temperature_2m_mean"],
    })
    df["month"] = df["date"].dt.month
    df["month_name"] = df["date"].dt.strftime("%B").str.capitalize()
    return df


def wmo_code_para_texto(codigo):
    """
    Converte o código WMO para uma descrição textual.
    """
    descricao = {
        0: "Céu limpo",
        1: "Principalmente limpo",
        2: "Parcialmente nublado",
        3: "Nublado",
        45: "Nevoeiro",
        48: "Nevoeiro com depósito",
        51: "Garoa fraca",
        53: "Garoa moderada",
        55: "Garoa densa",
        61: "Chuva fraca",
        63: "Chuva moderada",
        65: "Chuva forte",
        # Adicione outros códigos conforme necessário
    }
    return descricao.get(codigo, "Desconhecido")


def eh_tempo_bom_para_praia(wcode):
    """
    Retorna True se o código do tempo indicar condições boas para praia (0, 1 ou 2).
    """
    return wcode in [0, 1, 2]


def imprimir_feriados(df):
    """
    Imprime a lista de feriados com data e nome.
    """
    for _, row in df.iterrows():
        print(f"- {row['date'].strftime('%d/%m/%Y')}: {row['localName']}")


In [2]:
# Obter dados de feriados e clima
df_feriados = obter_feriados_brasil(2024)
df_clima = obter_clima_historico_rio("2024-01-01", "2024-07-31")

In [3]:
# ----------------------------------------------------------------------
# 1) Total e lista de feriados
# ----------------------------------------------------------------------
print("(1.a) Total de feriados em 2024 no Brasil:")
print(len(df_feriados))
print("\n(1.b) Lista de feriados em 2024:")
imprimir_feriados(df_feriados)

(1.a) Total de feriados em 2024 no Brasil:
15

(1.b) Lista de feriados em 2024:
- 01/01/2024: Confraternização Universal
- 12/02/2024: Carnaval
- 13/02/2024: Carnaval
- 29/03/2024: Sexta-feira Santa
- 31/03/2024: Domingo de Páscoa
- 21/04/2024: Dia de Tiradentes
- 01/05/2024: Dia do Trabalhador
- 30/05/2024: Corpus Christi
- 09/07/2024: Revolução Constitucionalista de 1932
- 07/09/2024: Dia da Independência
- 12/10/2024: Nossa Senhora Aparecida
- 02/11/2024: Dia de Finados
- 15/11/2024: Proclamação da República
- 20/11/2024: Dia da Consciência Negra
- 25/12/2024: Natal


In [99]:
# ----------------------------------------------------------------------
# 2) Mês com maior número de feriados
# ----------------------------------------------------------------------
contagem_mes = df_feriados["month_name"].value_counts()
mes_mais_feriados = contagem_mes.idxmax()
num_feriados_max = contagem_mes.max()
print(f"\n(2.a) Mês com maior número de feriados em 2024: {mes_mais_feriados} (com {num_feriados_max} feriados)")
print("\n(2.b) Feriados nesse mês:")
imprimir_feriados(df_feriados[df_feriados["month_name"] == mes_mais_feriados])


(2.a) Mês com maior número de feriados em 2024: Novembro (com 3 feriados)

(2.b) Feriados nesse mês:
- 02/11/2024: Dia de Finados
- 15/11/2024: Proclamação da República
- 20/11/2024: Dia da Consciência Negra


In [100]:
# ----------------------------------------------------------------------
# 3) Feriados em dias úteis (segunda a sexta)
# ----------------------------------------------------------------------
df_feriados["weekday_name"] = df_feriados["date"].dt.strftime("%A").str.capitalize()
feriados_uteis = df_feriados[df_feriados["weekday"] <= 4]
print(f"\n(3.a) Feriados em dias úteis (segunda a sexta): {len(feriados_uteis)}")
print("\n(3.b) Lista de feriados em dias úteis:")
for _, row in feriados_uteis.iterrows():
    print(f"- {row['date'].strftime('%d/%m/%Y')} ({row['weekday_name']}): {row['localName']}")


(3.a) Feriados em dias úteis (segunda a sexta): 10

(3.b) Lista de feriados em dias úteis:
- 01/01/2024 (Segunda-feira): Confraternização Universal
- 12/02/2024 (Segunda-feira): Carnaval
- 13/02/2024 (Terça-feira): Carnaval
- 29/03/2024 (Sexta-feira): Sexta-feira Santa
- 01/05/2024 (Quarta-feira): Dia do Trabalhador
- 30/05/2024 (Quinta-feira): Corpus Christi
- 09/07/2024 (Terça-feira): Revolução Constitucionalista de 1932
- 15/11/2024 (Sexta-feira): Proclamação da República
- 20/11/2024 (Quarta-feira): Dia da Consciência Negra
- 25/12/2024 (Quarta-feira): Natal


In [101]:
# ----------------------------------------------------------------------
# 4) Temperatura média mensal (01/01/2024 a 01/08/2024)
# ----------------------------------------------------------------------
temp_media_mensal = (df_clima.groupby(["month", "month_name"])["temp_mean"]
                        .mean().reset_index().sort_values("month"))
print("\n(4) Temperatura média (°C) por mês (jan a jul/2024):")
for _, row in temp_media_mensal.iterrows():
    print(f"    {row['month_name']}: {row['temp_mean']:.2f} °C")




(4) Temperatura média (°C) por mês (jan a jul/2024):
    Janeiro: 26.70 °C
    Fevereiro: 27.20 °C
    Março: 26.42 °C
    Abril: 25.17 °C
    Maio: 24.99 °C
    Junho: 22.64 °C
    Julho: 21.11 °C


In [102]:
# ----------------------------------------------------------------------
# 5) Tempo predominante por mês
# ----------------------------------------------------------------------
tempo_predominante = (df_clima.groupby(["month", "month_name", "weathercode"])
                        .size().reset_index(name="count"))
idx_max = tempo_predominante.groupby("month")["count"].idxmax()
tempo_por_mes = tempo_predominante.loc[idx_max, ["month", "month_name", "weathercode", "count"]]
tempo_por_mes["weather_description"] = tempo_por_mes["weathercode"].map(wmo_code_para_texto)
tempo_por_mes = tempo_por_mes.sort_values("month")
print("\n(5) Tempo predominante em cada mês (jan a jul/2024):")
for _, row in tempo_por_mes.iterrows():
    print(f"    {row['month_name']}: {row['weather_description']} ({row['count']} dias)")




(5) Tempo predominante em cada mês (jan a jul/2024):
    Janeiro: Chuva moderada (10 dias)
    Fevereiro: Garoa fraca (13 dias)
    Março: Garoa fraca (9 dias)
    Abril: Nublado (14 dias)
    Maio: Garoa fraca (10 dias)
    Junho: Nublado (12 dias)
    Julho: Nublado (12 dias)


In [103]:
# ----------------------------------------------------------------------
# 6) Tempo e temperatura em cada feriado (01/01/2024 a 01/08/2024)
# ----------------------------------------------------------------------
data_limite = pd.to_datetime("2024-08-01")
feriados_periodo = df_feriados[df_feriados["date"] < data_limite]
feriados_clima = pd.merge(feriados_periodo, df_clima, on="date", how="left")
feriados_clima["tempo_texto"] = feriados_clima["weathercode"].apply(wmo_code_para_texto)
print("\n(6) Tempo e temperatura média em cada feriado (01/01/2024 a 01/08/2024):")
for _, row in feriados_clima.iterrows():
    print(f"    {row['date'].strftime('%Y-%m-%d')} - {row['localName']}: "
            f"{row['tempo_texto']}, {row['temp_mean']:.2f} °C")


(6) Tempo e temperatura média em cada feriado (01/01/2024 a 01/08/2024):
    2024-01-01 - Confraternização Universal: Garoa fraca, 24.90 °C
    2024-02-12 - Carnaval: Nublado, 30.20 °C
    2024-02-13 - Carnaval: Garoa fraca, 30.50 °C
    2024-03-29 - Sexta-feira Santa: Chuva fraca, 25.00 °C
    2024-03-31 - Domingo de Páscoa: Garoa moderada, 24.50 °C
    2024-04-21 - Dia de Tiradentes: Parcialmente nublado, 23.10 °C
    2024-05-01 - Dia do Trabalhador: Céu limpo, 28.10 °C
    2024-05-30 - Corpus Christi: Garoa fraca, 20.80 °C
    2024-07-09 - Revolução Constitucionalista de 1932: Garoa moderada, 21.50 °C


In [104]:
# ----------------------------------------------------------------------
# 7) Feriados não aproveitáveis
# Critério: temp_mean < 20°C ou condições não favoráveis (weathercode fora de [0, 1, 2])
# ----------------------------------------------------------------------
def feriado_aproveitavel(row):
    if row["temp_mean"] < 20 or not eh_tempo_bom_para_praia(row["weathercode"]):
        return False
    return True

feriados_clima["aproveitavel"] = feriados_clima.apply(feriado_aproveitavel, axis=1)
nao_aproveitaveis = feriados_clima[~feriados_clima["aproveitavel"]]
if nao_aproveitaveis.empty:
    print("\n(7) Não houve feriado não aproveitável em 2024 nesse período.")
else:
    print("\n(7) Feriados NÃO aproveitáveis em 2024:")
    for _, row in nao_aproveitaveis.iterrows():
        print(f"    {row['date'].strftime('%Y-%m-%d')} - {row['localName']}: "
                f"{row['tempo_texto']}, {row['temp_mean']:.2f} °C")



(7) Feriados NÃO aproveitáveis em 2024:
    2024-01-01 - Confraternização Universal: Garoa fraca, 24.90 °C
    2024-02-12 - Carnaval: Nublado, 30.20 °C
    2024-02-13 - Carnaval: Garoa fraca, 30.50 °C
    2024-03-29 - Sexta-feira Santa: Chuva fraca, 25.00 °C
    2024-03-31 - Domingo de Páscoa: Garoa moderada, 24.50 °C
    2024-05-30 - Corpus Christi: Garoa fraca, 20.80 °C
    2024-07-09 - Revolução Constitucionalista de 1932: Garoa moderada, 21.50 °C


In [105]:
# ----------------------------------------------------------------------
# 8) Feriado mais aproveitável (maior temp_mean dentre os aproveitáveis)
# ----------------------------------------------------------------------
aproveitaveis = feriados_clima[feriados_clima["aproveitavel"]]
if aproveitaveis.empty:
    print("\n(8) Não há feriado aproveitável no período analisado.")
else:
    melhor_feriado = aproveitaveis.loc[aproveitaveis["temp_mean"].idxmax()]
    print(f"\n(8) Feriado mais aproveitável em 2024: "
            f"{melhor_feriado['date'].strftime('%Y-%m-%d')} - {melhor_feriado['localName']}.")
    print(f"    Tempo: {melhor_feriado['tempo_texto']}")
    print(f"    Temperatura média: {melhor_feriado['temp_mean']:.2f} °C")



(8) Feriado mais aproveitável em 2024: 2024-05-01 - Dia do Trabalhador.
    Tempo: Céu limpo
    Temperatura média: 28.10 °C
